# Cargar o conxunto de datos en MySQL (Notebook 1)

## Crea un notebook para este paso

Este arquivo (```1.CargarMySQL.ipynb```) xestiona este paso.

Tamén se achega o ficheiro ```books.csv``` coa base de datos de libros 
<br>(orixe: [Kaggle](https://www.kaggle.com/datasets/saurabhbagchi/books-dataset?resource=download))

## Conéctate á base de datos MySQL usando Python.

In [40]:
# Librarías que cómpre guindar

import mysql.connector
import csv

# Nomes e ferrollos da Conexión
HOST = "localhost"
USER = "root"
PASSWORD = "root_password" 
DATABASE = "data_pipeline_db"
CSV_FILE = "books.csv"          # Se está no mesmo directorio

# Conectamos coa nosa BD
try:
    mydb = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    cursor = mydb.cursor()

except mysql.connector.Error as err:
    print(f"Erro de MySQL: {err}")
    if '1049' in str(err):
         print("Non atopo a base de datos 'data_pipeline_db'.")

## Crea unha táboa en MySQL para gardar o conxunto de datos

In [21]:
# Creamos a táboa 'books' se non se fixo xa.
create_table_query = """
CREATE TABLE IF NOT EXISTS books (
        isbn VARCHAR(20) PRIMARY KEY,
        title VARCHAR(500) NOT NULL,
        author VARCHAR(255),
        year_of_publication INT,
        publisher VARCHAR(255)
    );
"""
cursor.execute(create_table_query)
print("Táboa 'books' creada, ou xa existía.")

Táboa 'books' creada, ou xa existía.


## Carga os datos do CSV na táboa

In [ ]:
# SISTEMA LENTO REVISADO

# --- CONFIGURACIÓN E CONEXIÓN (Debe estar nun bloque anterior e activo) ---
# Asume que mydb (conexión) e cursor (executor) están activos.
# Asume que CSV_FILE está definido e vale "books.csv".

insert_query = """
INSERT IGNORE INTO books (isbn, title, author, year_of_publication, publisher)
VALUES (%s, %s, %s, %s, %s)
""" 
BATCH_SIZE = 5000 
data_batch = []
filas_inseridas = 0

try:
    with open(CSV_FILE, 'r', encoding='ISO-8859-1') as file:
        csv_reader = csv.reader(file, delimiter=';') 
        next(csv_reader) # Saltar a cabeceira
        
        for row in csv_reader:
            if len(row) < 5: continue 
            
            # --- ⚠️ ELIMINAMOS O TRY/EXCEPT SILENCIOSO AQUÍ ---
            # O Python fallará aquí se o casting de int(row[3]) ou o mapeo de row[x] falla
            data_to_insert = (
                row[0],               
                row[1][:500],         # Aumentamos o límite a 500 para evitar Data Too Long
                row[2],               
                # Se row[3] non é un díxito e int(row[3]) falla, capturarase no 'except' principal
                int(row[3]) if row[3].isdigit() else 0, 
                row[4]                
            )
            data_batch.append(data_to_insert)

            # Executar o lote
            if len(data_batch) >= BATCH_SIZE:
                cursor.executemany(insert_query, data_batch)
                filas_inseridas += len(data_batch)
                data_batch = [] 

        # Executar o lote final
        if data_batch:
            cursor.executemany(insert_query, data_batch)
            filas_inseridas += len(data_batch)

    mydb.commit() # Commit final
    print(f"\nCarga de datos rematada. Total filas inseridas: {filas_inseridas}.")

except Exception as e:
    print(f"\nERRO CRÍTICO DE SQL/DATOS: {e}")
    if 'mydb' in locals() and mydb.is_connected():
        mydb.rollback() # Desfai calquera inserción incompleta


# Deixamos a conexión aberta para facer comprobacións (testing)



Carga de datos rematada. Total filas inseridas: 271379.


### Este sistema é moi lento. É mellor facer procesamento por lotes.



In [ ]:
# INSERIR POR LOTES (batch processing)




# Solicitude de inserción de datos 
insert_query = """
INSERT IGNORE INTO books (isbn, title, author, year_of_publication, publisher)
VALUES (%s, %s, %s, %s, %s)
""" 
BATCH_SIZE = 5000 # Arrebolamos por lotes para que remate máis axiña
data_batch = []
filas_inseridas = 0

try:
    # Asegúrate de que a conexión (mydb e cursor) foi establecida ANTES deste bloque
    
    with open(CSV_FILE, 'r', encoding='ISO-8859-1') as file:
        # Pasamos o valor de lindeira ("delimiter") xa que o separador de valores non é a coma
        csv_reader = csv.reader(file, delimiter=';') 
        next(csv_reader) # Saltar a cabeceira
        
        for row in csv_reader:
            if len(row) < 5: continue
            
            try:
                # 1. PREPARAR DATOS (igual que antes)
                data_to_insert = (
                    row[0],
                    row[1][:500],
                    row[2],
                    int(row[3]) if row[3].isdigit() else 0,
                    row[4]
                )
                data_batch.append(data_to_insert)

                # 2. EXECUTAR O LOTE CANDO SE ENCHE
                if len(data_batch) >= BATCH_SIZE:
                    cursor.executemany(insert_query, data_batch)
                    mydb.commit() # Commit periódico
                    filas_inseridas += len(data_batch)
                    data_batch = [] # Limpar o lote
                    print(f"Inseridos {filas_inseridas} rexistros...") # Indicador progreso

            except Exception:
                continue

        # 3. EXECUTAR O LOTE FINAL
        if data_batch:
            cursor.executemany(insert_query, data_batch)
            filas_inseridas += len(data_batch)

    mydb.commit() # Commit final
    print(f"\nCarregamento de datos rematado. Total filas inseridas: {filas_inseridas}.")

except FileNotFoundError:
    print(f"\nERRO: Non se atopa o ficheiro '{CSV_FILE}'.")

finally:
    if 'mydb' in locals() and mydb.is_connected():
        cursor.close()
        mydb.close()

KeyboardInterrupt: 

## Verifica que a carga foi correcta executando consultas SQL básicas

In [44]:

cursor.execute("SELECT COUNT(*) FROM books;")
count = cursor.fetchone()[0]
print(f"\nTotal de filas na táboa 'books': {count}")

# cursor.execute("SELECT isbn, title, author, year_of_publication, publisher FROM books LIMIT 5;")
cursor.execute("SELECT * FROM books LIMIT 5;")
print("\nListado das 5 primeiras  filas:")
# for (isbn, title, author, year_of_publication, publisher) in cursor:
#    print(f"\tISBN: {isbn}, Título: {title}, Autor: {author}, Ano: {year_of_publication}, Editorial: {publisher}")

for (isbn) in cursor:
   print(f"\tISBN: {isbn}, Título: {title}, Autor: {author}, Ano: {year_of_publication}, Editorial: {publisher}")




Total de filas na táboa 'books': 271065

Listado das 5 primeiras  filas:
	ISBN: ('0000913154', 'The Way Things Work: An Illustrated Encyclopedia of Technology', 'C. van Amerongen (translator)', 1967, 'Simon &amp; Schuster'), Título: T.S. Eliot Reading \The Wasteland\" and Other Poems", Autor: T.S. Eliot, Ano: 1993, Editorial: HarperCollins Publishers
	ISBN: ('0001010565', "Mog's Christmas", 'Judith Kerr', 1992, 'Collins'), Título: T.S. Eliot Reading \The Wasteland\" and Other Poems", Autor: T.S. Eliot, Ano: 1993, Editorial: HarperCollins Publishers
	ISBN: ('0001046438', 'Liar', 'Stephen Fry', 0, 'Harpercollins Uk'), Título: T.S. Eliot Reading \The Wasteland\" and Other Poems", Autor: T.S. Eliot, Ano: 1993, Editorial: HarperCollins Publishers
	ISBN: ('0001046713', 'Twopence to Cross the Mersey', 'Helen Forrester', 1992, 'HarperCollins Publishers'), Título: T.S. Eliot Reading \The Wasteland\" and Other Poems", Autor: T.S. Eliot, Ano: 1993, Editorial: HarperCollins Publishers
	ISBN: ('00

In [41]:



cursor_check = mydb_check.cursor()

    # Consulta para contar e mostrar un exemplo
    cursor_check.execute("SELECT COUNT(*) FROM libros;")
    count = cursor_check.fetchone()[0]
    print(f"\nTotal de filas na táboa 'libros': {count}")

    cursor_check.execute("SELECT title, author, average_rating FROM libros LIMIT 2;")
    print("\nExemplo das primeiras 2 filas:")
    for (title, author, rating) in cursor_check:
        print(f" - Título: {title}, Autor: {author}, Rating: {rating}")






create_table_query = """
CREATE TABLE IF NOT EXISTS books (
        isbn VARCHAR(20) PRIMARY KEY,
        title VARCHAR(500) NOT NULL,
        author VARCHAR(255),
        year_of_publication INT,
        publisher VARCHAR(255)
    );
"""
cursor.execute(create_table_query)
print("Táboa 'books' creada, ou xa existía.")












except mysql.connector.Error as err:
    print(f"Erro durante a verificación: {err}")



# Pechamo-la conexión

if 'mydb' in locals() and mydb.is_connected():
    cursor.close()
    mydb.close()



IndentationError: unexpected indent (62789149.py, line 4)